In [1]:
from mycode.common import *
SEPERATOR = "_TABLE_"

构造数据

In [2]:
from pandas import DataFrame

metadata, tables = fetch_data_from_sqlite(path='./mycode/1k_data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
result_table = None
x_table = ["BookLoan", "Book", "Library", "Student", "Enrollment", "Submission"]
x_key = [None, 'book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
x_how.insert(0, None)
result_table_columns_map = {}
# print(f"{len(result_table)=}")
original_tables = {
    key: tables[key] for key in x_table
}

In [3]:
set(tables["Book"].columns)

{'author', 'book_id', 'isbn', 'library_id', 'title'}

In [4]:
keys = set(x_key) - {None}
for i, x_table_name in enumerate(x_table):
    columns = set(tables[x_table_name].columns)
    result_table_columns_map[x_table_name] = {}
    unique_columns = columns - keys # 除去ref key
    reference_columns = keys.intersection(columns)
    temp = tables[x_table_name]
    if unique_columns:
        convert_columns = {x: x_table_name + SEPERATOR + x for x in unique_columns}
        temp = temp.rename(columns=convert_columns)
        result_table_columns_map[x_table_name].update({
            value: key for key, value in convert_columns.items()
        })
    if reference_columns:
        convert_columns = {x: "REF" + SEPERATOR + x for x in reference_columns}
        temp = temp.rename(columns=convert_columns)
        result_table_columns_map[x_table_name].update({
            value: key for key, value in convert_columns.items()
        })
    # x_table_name
    if result_table is not None:
        on_key = "REF" + SEPERATOR + x_key[i]
        result_table = pd.merge(result_table, temp, on=on_key, how=x_how[i])
    else:
        result_table = temp
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
#result_table

1000 1000 1000 1000 1014 1193 23


In [5]:
# result_table = result_table.sample(n=1000)
result_table

,BookLoan_TABLE_loan_id,REF_TABLE_book_id,REF_TABLE_student_id,BookLoan_TABLE_loan_date,BookLoan_TABLE_return_date,Book_TABLE_title,Book_TABLE_author,Book_TABLE_isbn,REF_TABLE_library_id,Library_TABLE_name,...,Student_TABLE_date_of_birth,Student_TABLE_major,Enrollment_TABLE_enrollment_id,Enrollment_TABLE_course_id,Enrollment_TABLE_semester,Enrollment_TABLE_grade,Submission_TABLE_submission_id,Submission_TABLE_assignment_id,Submission_TABLE_submission_date,Submission_TABLE_grade
0,4,4,260,2023-10-26,2025-09-03,Late rich.,Dr. Terri David,978-1-168-13965-8,4,"Williams, Moore and Kim Library",...,1994-07-14,734,309,967,Spring 2020,D,19,798,2022-04-01 16:19:43,C
1,6,6,944,2023-12-16,2025-03-22,Attack fish international.,Zachary Lawrence,978-1-02-166559-1,6,Ramirez-Randall Library,...,2003-01-18,935,500,425,Spring 2021,A,99,485,2022-01-31 04:46:38,B
2,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,79,876,2022-02-09 23:36:49,I
3,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,856,213,2022-08-04 10:27:59,F
4,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,972,668,2022-12-28 21:53:58,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,998,998,211,2023-11-09,2025-06-30,Gun land drive.,Sherry Hurst,978-1-4658-0306-1,998,Johnson Ltd Library,...,2006-01-23,812,372,334,Summer 2022,C,799,229,2022-04-06 19:02:31,I
1189,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,298,942,Summer 2019,None,106,497,2022-11-21 04:00:27,B
1190,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,298,942,Summer 2019,None,315,166,2022-04-05 07:47:40,I
1191,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,822,419,Winter 2020,B,106,497,2022-11-21 04:00:27,B


In [6]:
result_table_columns_map

{'BookLoan': {'BookLoan_TABLE_return_date': 'return_date',
  'BookLoan_TABLE_loan_date': 'loan_date',
  'BookLoan_TABLE_loan_id': 'loan_id',
  'REF_TABLE_student_id': 'student_id',
  'REF_TABLE_book_id': 'book_id'},
 'Book': {'Book_TABLE_author': 'author',
  'Book_TABLE_title': 'title',
  'Book_TABLE_isbn': 'isbn',
  'REF_TABLE_library_id': 'library_id',
  'REF_TABLE_book_id': 'book_id'},
 'Library': {'Library_TABLE_name': 'name',
  'Library_TABLE_location': 'location',
  'REF_TABLE_library_id': 'library_id'},
 'Student': {'Student_TABLE_name': 'name',
  'Student_TABLE_date_of_birth': 'date_of_birth',
  'Student_TABLE_major': 'major',
  'Student_TABLE_email': 'email',
  'REF_TABLE_student_id': 'student_id'},
 'Enrollment': {'Enrollment_TABLE_grade': 'grade',
  'Enrollment_TABLE_course_id': 'course_id',
  'Enrollment_TABLE_semester': 'semester',
  'Enrollment_TABLE_enrollment_id': 'enrollment_id',
  'REF_TABLE_student_id': 'student_id'},
 'Submission': {'Submission_TABLE_grade': 'grade'

检查metadata

In [7]:
result_table.to_csv("test_1k_label.csv", index=False)
dataset_csv = "test_1k_label.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-10-15 18:11:35.741 | INFO     | sdgx.data_models.metadata:from_dataloader:294 - Inspecting metadata...
2024-10-15 18:11:35.767 | DEBUG    | sdgx.manager:register:99 - Register for new model: subsetrelationshipinspector
2024-10-15 18:11:35.768 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailinspector
2024-10-15 18:11:35.768 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandidinspector
2024-10-15 18:11:35.769 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandmobilephoneinspector
2024-10-15 18:11:35.769 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandpostcode
2024-10-15 18:11:35.770 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandunifiedsocialcreditcode
2024-10-15 18:11:35.770 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandaddressinspector
2024-10-15 18:11:35.770 | DEBUG    | sdgx.manager:register:99 - Register for new mode

{'BookLoan_TABLE_loan_date',
 'BookLoan_TABLE_return_date',
 'Book_TABLE_author',
 'Book_TABLE_isbn',
 'Book_TABLE_title',
 'Enrollment_TABLE_grade',
 'Enrollment_TABLE_semester',
 'Library_TABLE_location',
 'Library_TABLE_name',
 'Student_TABLE_date_of_birth',
 'Student_TABLE_email',
 'Student_TABLE_name',
 'Submission_TABLE_grade',
 'Submission_TABLE_submission_date'}

In [8]:
loan_metadata.datetime_columns

{'BookLoan_TABLE_loan_date',
 'BookLoan_TABLE_return_date',
 'Student_TABLE_date_of_birth',
 'Submission_TABLE_submission_date'}

In [9]:
loan_metadata.column_list.__len__()

23

In [10]:
loan_metadata.datetime_format = {
    key: "%Y-%m-%d" for key in loan_metadata.datetime_columns if key != "submission_date" 
}
loan_metadata.datetime_format["submission_date"] = "%Y-%m-%d %H:%M:%S"
loan_metadata.datetime_format

{'Student_TABLE_date_of_birth': '%Y-%m-%d',
 'Submission_TABLE_submission_date': '%Y-%m-%d',
 'BookLoan_TABLE_loan_date': '%Y-%m-%d',
 'BookLoan_TABLE_return_date': '%Y-%m-%d',
 'submission_date': '%Y-%m-%d %H:%M:%S'}

In [11]:
loan_metadata.discrete_columns = set([
    key for key in loan_metadata.discrete_columns if key not in loan_metadata.datetime_columns
])
# loan_metadata.label_columns = loan_metadata.discrete_columns
#loan_metadata.label_columns.add("grade_x")
#loan_metadata.label_columns.add("grade_y")
print(f"{loan_metadata.int_columns=}\n{loan_metadata.float_columns=}\n{loan_metadata.const_columns=}\n{loan_metadata.bool_columns=}\n{loan_metadata.discrete_columns=}")

loan_metadata.int_columns={'Submission_TABLE_submission_id', 'REF_TABLE_student_id', 'REF_TABLE_book_id', 'Enrollment_TABLE_enrollment_id', 'Submission_TABLE_assignment_id', 'Enrollment_TABLE_course_id', 'BookLoan_TABLE_loan_id', 'REF_TABLE_library_id', 'Student_TABLE_major'}
loan_metadata.float_columns=set()
loan_metadata.const_columns=set()
loan_metadata.bool_columns=set()
loan_metadata.discrete_columns={'Book_TABLE_author', 'Library_TABLE_location', 'Book_TABLE_title', 'Student_TABLE_email', 'Book_TABLE_isbn', 'Library_TABLE_name', 'Student_TABLE_name', 'Submission_TABLE_grade', 'Enrollment_TABLE_semester', 'Enrollment_TABLE_grade'}


In [12]:
loan_metadata.column_encoder = {
    key: "label" for key in loan_metadata.discrete_columns
}

训练

In [13]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    metadata=loan_metadata,
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()

# synthesizer.save("test_model_100k")

2024-10-15 18:11:36.781 | DEBUG    | sdgx.manager:register:99 - Register for new model: outliertransformer
2024-10-15 18:11:36.781 | DEBUG    | sdgx.manager:register:99 - Register for new model: numericvaluetransformer
2024-10-15 18:11:36.782 | DEBUG    | sdgx.manager:register:99 - Register for new model: nonvaluetransformer
2024-10-15 18:11:36.782 | DEBUG    | sdgx.manager:register:99 - Register for new model: emptytransformer
2024-10-15 18:11:36.783 | DEBUG    | sdgx.manager:register:99 - Register for new model: constvaluetransformer
2024-10-15 18:11:36.784 | DEBUG    | sdgx.manager:register:99 - Register for new model: columnordertransformer
2024-10-15 18:11:36.784 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailgenerator
2024-10-15 18:11:36.785 | DEBUG    | sdgx.manager:register:99 - Register for new model: chnpiigenerator
2024-10-15 18:11:36.785 | DEBUG    | sdgx.manager:register:99 - Register for new model: intvalueformatter
2024-10-15 18:11:36.786 | DEBUG   

Preparing data:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-15 18:11:41.425 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column BookLoan_TABLE_loan_id...
2024-10-15 18:11:41.937 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column REF_TABLE_book_id...
2024-10-15 18:11:42.134 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column REF_TABLE_student_id...
2024-10-15 18:11:42.345 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column BookLoan_TABLE_loan_date...
2024-10-15 18:11:42.547 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column BookLoan_TABLE_return_date...
2024-10-15 18:11:42.739 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:122 - Fitting discrete column Book_TABLE_title...
2024-10-15 18:11:42.747 | DEBUG    | sdgx.models.components.o

[ColumnTransformInfo(column_name='BookLoan_TABLE_loan_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7), ColumnTransformInfo(column_name='REF_TABLE_book_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7), ColumnTransformInfo(column_name='REF_TABLE_student_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7), ColumnTransformInfo(column_name='BookLoan_TABLE_loan_date', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=7, activ

2024-10-15 18:11:54.220 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:302 - Starting model training, epochs: 1


Fitting batches:   0%|          | 0/2 [00:00<?, ?it/s]

2024-10-15 18:11:54.397 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:383 - Epoch 1, Loss G:  0.0475, Loss D:  0.0164, Time:  0.1763
2024-10-15 18:11:54.399 | INFO     | sdgx.models.ml.single_table.ctgan:fit:225 - CTGAN training finished.
2024-10-15 18:11:54.399 | INFO     | sdgx.synthesizer:fit:333 - Model fit... Finished


In [14]:
data_res : DataFrame = synthesizer.sample(1000)

2024-10-15 18:11:54.420 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

Sampling batches:   0%|          | 0/3 [00:00<?, ?it/s]

Inverse transforming:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-15 18:11:54.700 | INFO     | sdgx.data_processors.transformers.nan:reverse_convert:155 - Data reverse-converted by NonValueTransformer.
2024-10-15 18:11:54.702 | INFO     | sdgx.data_processors.transformers.outlier:reverse_convert:120 - Data reverse-converted by OutlierTransformer (No Action).
2024-10-15 18:11:54.925 | INFO     | sdgx.data_processors.formatters.int:reverse_convert:64 - Data reverse-converted by IntValueFormatter.
2024-10-15 18:11:54.926 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:165 - Data reverse-converting by DatetimeFormatter...
2024-10-15 18:11:54.926 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:167 - parameters : ['Student_TABLE_date_of_birth', 'Submission_TABLE_submission_date', 'BookLoan_TABLE_loan_date', 'BookLoan_TABLE_return_date'], {'Student_TABLE_date_of_birth': '%Y-%m-%d', 'Submission_TABLE_submission_date': '%Y-%m-%d', 'BookLoan_TABLE_loan_date': '%Y-%m-%d', 'BookLoan_TABLE_return_date': '%Y-%m-%

In [15]:
data_res.to_csv("test_1k_simulate.csv", index=False)

In [16]:
data_res

,BookLoan_TABLE_loan_id,REF_TABLE_book_id,REF_TABLE_student_id,BookLoan_TABLE_loan_date,BookLoan_TABLE_return_date,Book_TABLE_title,Book_TABLE_author,Book_TABLE_isbn,REF_TABLE_library_id,Library_TABLE_name,...,Student_TABLE_date_of_birth,Student_TABLE_major,Enrollment_TABLE_enrollment_id,Enrollment_TABLE_course_id,Enrollment_TABLE_semester,Enrollment_TABLE_grade,Submission_TABLE_submission_id,Submission_TABLE_assignment_id,Submission_TABLE_submission_date,Submission_TABLE_grade
0,134,913,155,2024-04-03,2025-10-29,Power indeed do.,Stephen Steele,978-1-02-424817-3,566,Jefferson-James Library,...,2004-02-24,36,276,-15,Spring 2022,,606,568,1970-01-01,D
1,747,365,676,2024-07-11,2025-02-19,Edge through control.,Heather Howell,978-1-69631-792-4,89,Smith-Anderson Library,...,1995-05-16,465,-160,965,Spring 2022,F,872,137,1970-01-01,C
2,913,423,203,2024-07-11,2025-08-23,However more.,Laura Williams,978-1-09-440634-3,463,Hayes-Griffith Library,...,1996-08-12,662,643,657,Summer 2022,C,664,711,1970-01-01,C
3,804,245,611,2023-12-30,2024-11-21,Decision little feel.,Jason Holland,978-1-64140-905-6,936,"Mason, Silva and Johnson Library",...,2004-09-08,441,21,442,Spring 2019,D,361,868,1970-01-01,C
4,598,497,190,2024-09-28,2025-01-12,Important lead when.,Lauren Brewer,978-0-8426-0778-0,564,"Martinez, Rosales and Kelly Library",...,1996-07-17,718,-20,327,Spring 2019,D,416,459,1970-01-01,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,777,1006,437,2024-09-24,2025-12-07,Future through ever if.,David Robinson,978-1-277-89857-6,435,Merritt LLC Library,...,1997-03-14,61,704,420,Summer 2022,D,463,479,1970-01-01,C
996,178,405,304,2024-11-11,2024-12-19,Local fight.,Christy Hutchinson,978-0-256-85205-9,444,"Pollard, Adams and Watts Library",...,2002-12-02,965,27,257,Winter 2019,D,1012,159,1970-01-01,A
997,414,707,574,2024-05-07,2025-04-14,Prepare investment probably they.,Lauren Perkins,978-0-544-42598-9,396,Miller-Moreno Library,...,1997-06-26,656,588,561,Summer 2020,,400,640,1970-01-01,D
998,924,809,1016,2024-04-12,2024-11-06,Hold me.,Norman Lee,978-0-7826-0092-6,241,Lee-Stark Library,...,2001-01-10,907,730,338,Summer 2023,D,661,308,1970-01-01,B


In [17]:
synthesized_tables = {
    table_name: data_res[table_map.keys()].rename(columns=table_map) for table_name, table_map in result_table_columns_map.items()
}

In [18]:
synthesized_tables["Book"]

,author,title,isbn,library_id,book_id
0,Stephen Steele,Power indeed do.,978-1-02-424817-3,566,913
1,Heather Howell,Edge through control.,978-1-69631-792-4,89,365
2,Laura Williams,However more.,978-1-09-440634-3,463,423
3,Jason Holland,Decision little feel.,978-1-64140-905-6,936,245
4,Lauren Brewer,Important lead when.,978-0-8426-0778-0,564,497
...,...,...,...,...,...
995,David Robinson,Future through ever if.,978-1-277-89857-6,435,1006
996,Christy Hutchinson,Local fight.,978-0-256-85205-9,444,405
997,Lauren Perkins,Prepare investment probably they.,978-0-544-42598-9,396,707
998,Norman Lee,Hold me.,978-0-7826-0092-6,241,809


In [22]:
def save_xlsx(tables, name):
    # 使用ExcelWriter保存为多sheet的Excel文件
    with pd.ExcelWriter(name) as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [20]:
def read_xlsx(name):
    # 读取多sheet的Excel文件
    sheets_dict = pd.read_excel(name, sheet_name=None)
    return sheets_dict

In [23]:
save_xlsx(synthesized_tables, "test_1k_simulate_split.xlsx")
save_xlsx(original_tables, "test_1k_label_split.xlsx")